<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/algoritmos_gen%C3%A9ticos/optimizaci%C3%B3n_combinatoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción

Un algoritmo genético es una técnica de optimización que se inspira en la evolución biológica para encontrar soluciones a problemas complejos. En términos generales, consiste en crear una población inicial de soluciones candidatas, aplicar operaciones de selección, cruza y mutación para generar nuevas soluciones, y evaluar el rendimiento de cada solución en base a una función objetivo definida. Las soluciones con mejores rendimientos tienen más posibilidades de sobrevivir y reproducirse en la siguiente generación, mientras que las soluciones peor adaptadas son eliminadas.

La idea detrás de los algoritmos genéticos es que, al aplicar una selección natural sobre soluciones candidatas, se pueden encontrar soluciones mejores y mejores con el tiempo. En la inteligencia artificial, los algoritmos genéticos se utilizan a menudo en problemas de optimización, como encontrar la mejor solución para un problema de programación lineal o de asignación de recursos. También se pueden utilizar en la generación de soluciones creativas en áreas como la arquitectura o el diseño.

**Objetivo** En este ejemplo, veremos como implementar el problema de la mochila, también conocido como problema de la mochila 0-1 o problema Knapsack. Es un problema de optimización combinatoria. El problema consiste en seleccionar un subconjunto de elementos de un conjunto dado, de tal manera que la suma de sus pesos no exceda una capacidad dada, y que la suma de sus valores sea lo más grande posible.

El problema se puede formular de la siguiente manera: se tiene una mochila con una capacidad máxima de peso W, y se tiene un conjunto de n elementos, cada uno con un valor v y un peso w. El objetivo es seleccionar un subconjunto de elementos cuya suma de pesos no exceda W, y cuya suma de valores sea lo más grande posible.

El problema de la mochila es un problema NP-completo, lo que significa que no se conoce un algoritmo eficiente para resolverlo en el peor de los casos. Sin embargo, existen varios algoritmos heurísticos y metaheurísticos que pueden proporcionar soluciones aproximadas al problema de manera eficiente en la mayoría de los casos.

In [ ]:
from functools import partial
from collections import namedtuple
from random import choices, randint, randrange, random

In [ ]:
# Función para generar un genoma aleatorio de longitud "length"
def generate_genome(length):
    # Se utiliza la función "choices" del módulo "random" para generar un genoma aleatorio
    return choices([0, 1], k=length)

In [ ]:
# Función para generar una población aleatoria de tamaño "size", donde cada individuo tiene un genoma de longitud "genome_length"
def generate_population(size, genome_length):
    # Se utiliza la función "generate_genome" para generar cada individuo de la población
    return [generate_genome(genome_length) for _ in range(size)]

In [ ]:
# Función para realizar un cruce de un punto entre dos genomas "a" y "b"
def single_point_crossover(a, b):
    # Se verifica que los genomas "a" y "b" tengan la misma longitud
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")
    # Se obtiene la longitud de los genomas
    length = len(a)
    # Si la longitud es menor a 2, se devuelve los mismos genomas
    if length < 2:
        return a, b
    # Se elige un punto de cruce aleatorio entre 1 y la longitud del genoma menos 1
    p = randint(1, length - 1)
    # Se intercambian las partes del genoma a partir del punto de cruce para generar dos nuevos genomas
    return a[0:p] + b[p:], b[0:p] + a[p:]

In [ ]:
# Función para mutar un genoma. "num" indica la cantidad de mutaciones a realizar, y "probability" es la probabilidad de que un bit sea invertido.
def mutation(genome, num=1, probability=0.5):
    # Se realizan "num" mutaciones en el genoma
    for _ in range(num):
        # Se elige un índice aleatorio del genoma
        index = randrange(len(genome))
        # Se invierte el bit en el índice elegido con una probabilidad "probability"
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    # Se devuelve el genoma mutado
    return genome

In [ ]:
# Función para calcular el fitness total de una población
def population_fitness(population, fitness_func):
    # Se utiliza la función "sum" para calcular la suma de los fitness de cada individuo de la población
    return sum([fitness_func(genome) for genome in population])

In [ ]:
# Función para seleccionar una pareja de individuos para el cruce
def selection_pair(population, fitness_func):
    # Se utiliza la función "choices" del módulo "random" para elegir dos individuos de la población con una probabilidad ponderada por su fitness
    return choices(population=population, weights=[fitness_func(gene) for gene in population], k=2)

In [ ]:
# Función para ordenar una población de mayor a menor fitness
def sort_population(population, fitness_func):
    # Se utiliza la función "sorted" para ordenar la población en función de su fitness, de mayor a menor
    return sorted(population, key=fitness_func, reverse=True)

In [ ]:
# Función para convertir un genoma a una cadena de caracteres
def genome_to_string(genome):
    # Se utiliza la función "join" para concatenar los elementos del genoma como una cadena de caracteres
    return "".join(map(str, genome))

In [ ]:
def print_stats(population, generation_id, fitness_func):
    print("Generación %02d" % generation_id)
    print("=============")
    print("Población: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Ajuste medio: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print("Mejor: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Peor: %s (%f)" % (genome_to_string(sorted_population[-1]), fitness_func(sorted_population[-1])))
    print("")
    return sorted_population[0]

In [ ]:
# Función para ejecutar el algoritmo evolutivo
def run_evolution(populate_func, fitness_func, fitness_limit, selection_func=selection_pair, crossover_func=single_point_crossover,
                  mutation_func=mutation, generation_limit=100, printer=print_stats):
    
    # Se crea la población inicial
    population = populate_func()

    # Se ejecutan las generaciones del algoritmo
    for i in range(generation_limit):
        # Se ordena la población en función de su fitness
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        # Se imprime la información de la población si se proporciona una función de impresión
        if printer is not None:
            printer(population, i, fitness_func)

        # Si se alcanza el fitness límite, se sale del bucle
        if fitness_func(population[0]) >= fitness_limit:
            break

        # Se seleccionan los dos individuos con mayor fitness para la siguiente generación
        next_generation = population[0:2]

        # Se generan los descendientes a partir de los padres seleccionados mediante cruce y mutación
        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        # Se actualiza la población para la siguiente generación
        population = next_generation

    # Se devuelve la población final y el número de generaciones ejecutadas
    return population, i

In [ ]:
Thing = namedtuple('Thing', ['name', 'value', 'weight'])

In [ ]:
def generate_things(num):
    # Genera una lista de objetos "Thing" con valores y pesos aleatorios.
    return [Thing(f"thing{i}", i, i) for i in range(1, num+1)]

In [ ]:
def fitness(genome, things, weight_limit):
    if len(genome) != len(things):
        raise ValueError("el genoma y los objetos deben ser de la misma longitud")

    weight = 0
    value = 0
    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value

            if weight > weight_limit:
                return 0

    return value

In [ ]:
first_example = [
    Thing('Laptop', 500, 2200),
    Thing('Audífonos', 150, 160),
    Thing('Taza de café', 60, 350),
    Thing('Cuaderno', 40, 333),
    Thing('Botella de agua', 30, 192),
]

second_example = first_example + [
    Thing('Dulces', 5, 25),
    Thing('Calcetines', 10, 38),
    Thing('Pañuelos', 15, 80),
    Thing('Celular', 500, 200),
    Thing('Batería', 100, 70)
]

In [ ]:
things = generate_things(22)
things = second_example

weight_limit = 3000

print("Peso límite: %dkg" % weight_limit)

Peso límite: 3000kg


In [ ]:
def bruteforce(things, weight_limit: int):
    if len(things) == 0:
        return 0, []

    max_value = 0
    max_valued_packed = []
    for i, thing in enumerate(things):
        if thing.weight > weight_limit:
            continue

        value, packed = bruteforce(things[i + 1:], weight_limit - thing.weight)
        if value + thing.value >= max_value:
            max_value = value + thing.value
            max_valued_packed = [thing] + packed

    return max_value, max_valued_packed

In [ ]:
result = bruteforce(things, weight_limit)
result

(1310,
 [Thing(name='Laptop', value=500, weight=2200),
  Thing(name='Audífonos', value=150, weight=160),
  Thing(name='Botella de agua', value=30, weight=192),
  Thing(name='Dulces', value=5, weight=25),
  Thing(name='Calcetines', value=10, weight=38),
  Thing(name='Pañuelos', value=15, weight=80),
  Thing(name='Celular', value=500, weight=200),
  Thing(name='Batería', value=100, weight=70)])

In [ ]:
population, generations = run_evolution(
		populate_func=partial(generate_population, size=10, genome_length=len(things)),
		fitness_func=partial(fitness, things=things, weight_limit=weight_limit),
		fitness_limit=result[0],
		generation_limit=100)

Generación 00
Población: [0110100011, 0100101110, 1001011101, 0001000111, 0001001010, 0011100101, 0011110000, 0000111100, 1110111101, 1011110010]
Ajuste medio: 386.000000
Mejor: 0110100011 (840.000000)
Peor: 1011110010 (0.000000)

Generación 01
Población: [1100101111, 0110100111, 0110100011, 0110100011, 0100101110, 0010100011, 0011001010, 0111100101, 0001100101, 1110100010]
Ajuste medio: 642.500000
Mejor: 1100101111 (1305.000000)
Peor: 1110100010 (0.000000)

Generación 02
Población: [1100101111, 1000101011, 0111100011, 0110100111, 0110100011, 0110100011, 0010101011, 0010100011, 0000100111, 0110100001]
Ajuste medio: 823.500000
Mejor: 1100101111 (1305.000000)
Peor: 0110100001 (340.000000)

Generación 03
Población: [1100101111, 1000101011, 1000100011, 0110101011, 0110100011, 0011100011, 0010100011, 0000001011, 0000100101, 1010101011]
Ajuste medio: 744.000000
Mejor: 1100101111 (1305.000000)
Peor: 1010101011 (0.000000)

Generación 04
Población: [1100101111, 1100101011, 1000101111, 100010101

In [ ]:
def from_genome(genome, things):
    result = []
    for i, thing in enumerate(things):
        if genome[i] == 1:
            result += [thing]

    return result

In [ ]:
def to_string(things):
    return f"[{', '.join([t.name for t in things])}]"


def value(things):
    return sum([t.value for t in things])


def weight(things):
    return sum([p.weight for p in things])

In [ ]:
def print_stats2(things):
    print(f"Things: {to_string(things)}")
    print(f"Value {value(things)}")
    print(f"Weight: {weight(things)}")

In [ ]:
sack = from_genome(population[0], things)
print_stats2(sack)

Things: [Laptop, Audífonos, Botella de agua, Dulces, Calcetines, Pañuelos, Celular, Batería]
Value 1310
Weight: 2965


In [ ]:
result

(1310,
 [Thing(name='Laptop', value=500, weight=2200),
  Thing(name='Audífonos', value=150, weight=160),
  Thing(name='Botella de agua', value=30, weight=192),
  Thing(name='Dulces', value=5, weight=25),
  Thing(name='Calcetines', value=10, weight=38),
  Thing(name='Pañuelos', value=15, weight=80),
  Thing(name='Celular', value=500, weight=200),
  Thing(name='Batería', value=100, weight=70)])

**Referencias**

* https://arpitbhayani.me/blogs/genetic-knapsack
* https://plainenglish.io/blog/genetic-algorithm-in-python-101-da1687d3339b
* https://www.kdnuggets.com/2023/01/knapsack-problem-genetic-programming-python.html

**Por hacer**

* Terminar comentarios a las funciones
* Reestructurar orden de funciones